In [1]:
# variables
train_en = './dataset/train.en'
train_vi = './dataset/train.vi'

In [2]:
# read the translation files
datafile_en = open(train_en, 'r')
datafile_vi = open(train_vi, 'r')

In [3]:
# get all sentences from the files
sentences_en = datafile_en.readlines()
sentences_vi = datafile_vi.readlines()

In [4]:
# play with data
print(len(sentences_en))
print(len(sentences_vi))
print(sentences_en[:5])
print(sentences_vi[:5])

133317
133317
['Rachel Pike : The science behind a climate headline\n', 'In 4 minutes , atmospheric chemist Rachel Pike provides a glimpse of the massive scientific effort behind the bold headlines on climate change , with her team -- one of thousands who contributed -- taking a risky flight over the rainforest in pursuit of data on a key molecule .\n', 'I &apos;d like to talk to you today about the scale of the scientific effort that goes into making the headlines you see in the paper .\n', 'Headlines that look like this when they have to do with climate change , and headlines that look like this when they have to do with air quality or smog .\n', 'They are both two branches of the same field of atmospheric science .\n']
['Khoa học đằng sau một tiêu đề về khí hậu\n', 'Trong 4 phút , chuyên gia hoá học khí quyển Rachel Pike giới thiệu sơ lược về những nỗ lực khoa học miệt mài đằng sau những tiêu đề táo bạo về biến đổi khí hậu , cùng với đoàn nghiên cứu của mình -- hàng ngàn người đã cố

In [5]:
# create a character array of all the english characters
char_array_en = []
for sentence in sentences_en:
    for char in sentence:
        char_array_en.append(char)

In [6]:
# create a character array of all the vietnamese characters
char_array_vi = []
for sentence in sentences_vi:
    for char in sentence:
        char_array_vi.append(char)

In [7]:
# create a dictionary of words and their frequency for english characters
from collections import Counter
char_dict_en = dict(Counter(char_array_en))
len(char_dict_en)

124

In [8]:
# create a dictionary of words and their frequency for vietnamese characters
from collections import Counter
char_dict_vi = dict(Counter(char_array_vi))
len(char_dict_vi)

238